In [ ]:
# Load all imports

import os
from pathlib import Path
from PIL import Image
import numpy as np
import numpy.typing as npt
import pandas as pd

Create coral mask for bleaching data

In [ ]:
# Define the directories
root_dir = Path("data/coral_bleaching")
images_dir = root_dir / "images"
bleached_dir = root_dir / "masks_bleached"
non_bleached_dir = root_dir / "masks_non_bleached"
out_dir = root_dir / "masks_coral"
out_dir.mkdir(parents=True, exist_ok=True)

def load_mask(path: Path) -> npt.NDArray[np.uint8]:
    """
    Loads the mask for a given path

    Args:
        path (Path): Path of the mask

    Returns:
        npt.NDArray[np.uint8]: Array with the actual mask.
    """
    mask = Image.open(path).convert("L") # Load mask and convert to greyscale
    return np.array(mask).astype(np.uint8)

# Loop over all images, and create the coral masks
created = fails = 0
for img_path in sorted(images_dir.glob("*.jpg")):
    stem = img_path.stem
    bleached_path = bleached_dir / f"{stem}_bleached.png"
    non_bleached_path = non_bleached_dir / f"{stem}_non_bleached.png"
    
    mask_bleached = load_mask(bleached_path)
    mask_non_bleached = load_mask(non_bleached_path)
    mask_coral = (mask_bleached | mask_non_bleached).astype(np.uint8)

    output_path = out_dir / f"{stem}.png"
    Image.fromarray((mask_coral).astype(np.uint8)).save(output_path)

Create dataframe from bleaching dataset. Remove masks with less than 1% coverage

In [75]:
# Load all datasets, and remove any coral masks that have less than 1% coverage.

root_dir = Path("data/coral_bleaching")
images_dir = root_dir / "images"
coral_dir = root_dir / "masks_coral"

rows = []

for img_path in sorted(images_dir.glob("*.jpg")):
    stem = img_path.stem
    with Image.open(img_path) as im:
        img_W, img_H = im.size
  
    paths = {
        "image": img_path,
        "mask_bleached": bleached_dir / f"{stem}_bleached.png",
        "mask_non_bleached": non_bleached_dir / f"{stem}_non_bleached.png",
        "mask_coral": coral_dir / f"{stem}.png",
    }
    
    array = np.array(Image.open(paths["mask_coral"]).convert("L")).astype(np.uint8) / 255
    coral_mask_H, coral_mask_W = array.shape
    coral_coverage = array.sum() / (coral_mask_H * coral_mask_W)
    if coral_coverage >= 0.01:
        rows.append({
        "stem": stem,
        "image_path": str(paths["image"]),
        "mask_coral_path": str(paths["mask_coral"]) if paths["mask_coral"].exists() else None,
        "coral_coverage": round(float(coral_coverage), 3),
        "source": "coral_bleaching_reefsupport", 
    })

df_clean = pd.DataFrame(rows)

KeyboardInterrupt: 

In [ ]:
# Define the directories
root_dir = Path("data/coral_bleaching")
images_dir = root_dir / "images"
bleached_dir = root_dir / "masks_bleached"
non_bleached_dir = root_dir / "masks_non_bleached"
out_dir = root_dir / "masks_coral"
out_dir.mkdir(parents=True, exist_ok=True)

# Loop over all images, and create the coral masks
created = fails = 0
for img_path in sorted(images_dir.glob("*.jpg")):
    stem = img_path.stem
    bleached_path = bleached_dir / f"{stem}_bleached.png"
    non_bleached_path = non_bleached_dir / f"{stem}_non_bleached.png"
    
    mask_bleached = np.array(Image.open(bleached_path).convert("L")).astype(np.uint8)
    mask_non_bleached = np.array(Image.open(non_bleached_path).convert("L")).astype(np.uint8)
    mask_coral = (mask_bleached | mask_non_bleached).astype(np.uint8)

    output_path = out_dir / f"{stem}.png"
    Image.fromarray((mask_coral).astype(np.uint8)).save(output_path)

In [ ]:
root_dir = Path("data/benthic_datasets")


for site in sorted(os.listdir(root_dir)):
    images_dir = root_dir / site / "images"
    coral_stitched_dir = root_dir / site / "masks_stitched"
    coral_dir = root_dir / site / "masks_coral"
    out_dir.mkdir(parents=True, exist_ok=True)
    rows = []

    for img_path in sorted([*images_dir.glob("*.jpg"), *images_dir.glob("*.JPG")]):
        stem = img_path.stem
        with Image.open(img_path) as im:
            img_W, img_H = im.size
    
        paths = {
            "image": img_path,
            "mask_coral": coral_dir / f"{stem}_mask.png",
        }
        
        array = np.array(Image.open(paths["mask_coral"]).convert("RGBA")).astype(np.uint8)
        print(img_path)
        print(np.unique(array))
        # coral_mask_H, coral_mask_W = array.shape
        coral_coverage = array.sum() / (coral_mask_H * coral_mask_W)
        if coral_coverage >= 0.01:
            rows.append({
            "stem": stem,
            "image_path": str(paths["image"]),
            "mask_coral_path": str(paths["mask_coral"]) if paths["mask_coral"].exists() else None,
            "coral_coverage": round(float(coral_coverage), 3),
            "source": "coral_bleaching_reefsupport", 
        })




data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (103).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (103).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (104).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (104).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (110).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (110).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (111).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (111).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (113).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (113).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 (118).JPG
[  0 255]
data\benthic_datasets\SEAFLOWER_BOLIVAR\images\20220912_AnB_CB10 

KeyboardInterrupt: 